# Simulating Stock Trading Strategies with SMA and RSI Indicators

In [13]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import tensorflow as tf

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG

## Data Preprocessing

In [16]:
# Data Collection
ticker = 'AAPL'
stock_data = yf.download(ticker, start='2020-01-01', end='2024-01-01')
stock_data.info()

[*********************100%%**********************]  1 of 1 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1006 entries, 2020-01-02 to 2023-12-29
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1006 non-null   float64
 1   High       1006 non-null   float64
 2   Low        1006 non-null   float64
 3   Close      1006 non-null   float64
 4   Adj Close  1006 non-null   float64
 5   Volume     1006 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 55.0 KB


In [18]:
stock_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Open,1006.0,1.406755e+02,3.331002e+01,5.702000e+01,1.236825e+02,1.455400e+02,1.663025e+02,1.980200e+02
High,1006.0,1.423214e+02,3.343057e+01,5.712500e+01,1.250300e+02,1.472650e+02,1.681475e+02,1.996200e+02
Low,1006.0,1.391435e+02,3.317920e+01,5.315250e+01,1.221575e+02,1.441200e+02,1.648150e+02,1.970000e+02
Close,1006.0,1.408081e+02,3.331386e+01,5.609250e+01,1.235925e+02,1.458600e+02,1.662150e+02,1.981100e+02
Adj Close,1006.0,1.388642e+02,3.357695e+01,5.456973e+01,1.211879e+02,1.437545e+02,1.642670e+02,1.973611e+02
Volume,1006.0,9.895211e+07,5.439653e+07,2.404830e+07,6.407675e+07,8.467540e+07,1.155069e+08,4.265100e+08


## BackTesting Environment (2015 - 2019)

In [21]:
class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)
    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()
            
historical_data = yf.download('AAPL', start='2015-01-01', end='2019-01-01')
backtest = Backtest(historical_data, SmaCross, commission=.002, exclusive_orders=True)
stats = backtest.run()

backtest.plot(filename='backtest_results.html', open_browser=True)

[*********************100%%**********************]  1 of 1 completed
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/opt/anaconda3/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/anaconda3/lib/python3.12/site-packages/b

GridPlot(id='p1325', ...)

## Buy/Sell Decision Based on Stock Closing Prices

In [43]:
import time
import numpy as np
import datetime as dt
import yfinance as yf

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.maximum(delta, 0)
    loss = np.abs(np.minimum(delta, 0))

    avg_gain = np.mean(gain[:period])
    avg_loss = np.mean(loss[:period])

    rsi = []
    for i in range(period, len(prices)):
        avg_gain = (avg_gain * (period - 1) + gain[i - period]) / period
        avg_loss = (avg_loss * (period - 1) + loss[i - period]) / period
        rs = avg_gain / avg_loss if avg_loss != 0 else 0
        rsi_value = 100 - (100 / (1 + rs))
        rsi.append(rsi_value)

    return rsi

def trade_yfinance():
    ticker = 'AAPL'
    data = yf.download(ticker, period='1mo', interval='1h')  # Download 1 month of hourly data
    close_prices = data['Close'].tolist()  # Convert the close prices to a list for simulation

    # Parameters for SMA calculation
    sma_short_period = 10
    sma_long_period = 20
    rsi_period = 14

    # Stop-loss and take-profit parameters (percentage)
    stop_loss_pct = 0.02  # 2% stop loss
    take_profit_pct = 0.04  # 4% take profit

    # Variables for simulation
    current_position = None
    cash = 100000  # Initial virtual cash
    stock_qty = 0
    entry_price = 0
    start_time = dt.datetime.now()
    max_duration = dt.timedelta(minutes=5)  # Simulate for 5 minutes
    index = 0
    total_profit = 0  # Track total profit or loss

    while True:
        # Check if the script has run for longer than the max duration
        if dt.datetime.now() - start_time > max_duration:
            print("Stopping the script after 5 minutes.")
            break

        print("Running simulation loop...")

        # Get the current price
        if index < len(close_prices):
            current_price = close_prices[index]
            index += 1
        else:
            print("End of historical price data.")
            break

        # Extract Close Prices for SMA Calculation (take the last `sma_long_period` prices)
        if index >= sma_long_period:
            recent_prices = close_prices[index - sma_long_period:index]

            # Calculate Moving Averages
            sma_short = np.mean(recent_prices[-sma_short_period:])
            sma_long = np.mean(recent_prices)

            # Calculate RSI
            if index >= sma_long_period + rsi_period:
                rsi_values = calculate_rsi(close_prices[index - (sma_long_period + rsi_period):index], rsi_period)
                rsi = rsi_values[-1]
            else:
                continue

            # Debug information for moving averages and RSI
            print(f"SMA Short: {sma_short}, SMA Long: {sma_long}, RSI: {rsi}")

            # Buy Signal with Relaxed RSI Confirmation
            if sma_short > sma_long and current_position is None and rsi < 40:
                qty = int(cash // current_price)
                if qty > 0:
                    stock_qty = qty
                    cash -= stock_qty * current_price
                    entry_price = current_price
                    current_position = 'long'
                    print(f"Bought {stock_qty} shares at {current_price}, Remaining Cash: {cash}")

            # Stop-loss or Take-profit check
            if current_position == 'long':
                # Stop-loss triggered
                if current_price <= entry_price * (1 - stop_loss_pct):
                    cash += stock_qty * current_price
                    profit = (current_price - entry_price) * stock_qty
                    total_profit += profit
                    print(f"Stop-Loss Triggered: Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
                    stock_qty = 0
                    current_position = None

                # Take-profit triggered
                elif current_price >= entry_price * (1 + take_profit_pct):
                    cash += stock_qty * current_price
                    profit = (current_price - entry_price) * stock_qty
                    total_profit += profit
                    print(f"Take-Profit Triggered: Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
                    stock_qty = 0
                    current_position = None

            # Sell Signal with Relaxed RSI Confirmation
            elif sma_short < sma_long and current_position == 'long' and rsi > 60:
                cash += stock_qty * current_price
                profit = (current_price - entry_price) * stock_qty
                total_profit += profit
                print(f"Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
                stock_qty = 0
                current_position = None

            # Debug profit/loss tracking
            print(f"Total Profit/Loss so far: {total_profit}")

        time.sleep(1)

    # Final evaluation of performance
    if current_position == 'long':
        # Sell any remaining stock at the last available price
        cash += stock_qty * current_price
        profit = (current_price - entry_price) * stock_qty
        total_profit += profit
        print(f"Final Sale: Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
        stock_qty = 0
        current_position = None

    print(f"Total Profit/Loss at the end of simulation: {total_profit}")

trade_yfinance()


[*********************100%%**********************]  1 of 1 completed


Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
Running simulation loop...
SMA Short: 226.17301025390626, SMA Long: 226.60627975463868, RSI: 39.4912014066910

## Buy/Sell Decision Using Full Stock Data

In [23]:
import time
import numpy as np
import datetime as dt
import yfinance as yf

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.maximum(delta, 0)
    loss = np.abs(np.minimum(delta, 0))

    avg_gain = np.mean(gain[:period])
    avg_loss = np.mean(loss[:period])

    rsi = []
    for i in range(period, len(prices)):
        avg_gain = (avg_gain * (period - 1) + gain[i - period]) / period
        avg_loss = (avg_loss * (period - 1) + loss[i - period]) / period
        rs = avg_gain / avg_loss if avg_loss != 0 else 0
        rsi_value = 100 - (100 / (1 + rs))
        rsi.append(rsi_value)

    return rsi

def trade_finance():
    ticker = 'AAPL'
    data = yf.download(ticker, period='1mo', interval='1h')  # Download 1 month of hourly data

    # Columns to use
    columns = ["High", "Low", "Open", "Close", "Volume", "Adj Close"]
    selected_data = data[columns].to_numpy()  # Convert selected columns to NumPy array

    # Extract individual columns
    high_prices = selected_data[:, 0]  # High prices
    low_prices = selected_data[:, 1]  # Low prices
    open_prices = selected_data[:, 2]  # Open prices
    close_prices = selected_data[:, 3]  # Close prices

    # Parameters for SMA calculation
    sma_short_period = 10
    sma_long_period = 20
    rsi_period = 14

    # Stop-loss and take-profit parameters (percentage)
    stop_loss_pct = 0.02  # 2% stop loss
    take_profit_pct = 0.04  # 4% take profit

    # Variables for simulation
    current_position = None
    cash = 100000  # Initial virtual cash
    stock_qty = 0
    entry_price = 0
    start_time = dt.datetime.now()
    max_duration = dt.timedelta(minutes=5)  # Simulate for 5 minutes
    index = 0
    total_profit = 0  # Track total profit or loss

    while True:
        # Check if the script has run for longer than the max duration
        if dt.datetime.now() - start_time > max_duration:
            print("Stopping the script after 5 minutes.")
            break

        # Get the current price
        if index < len(close_prices):
            current_price = close_prices[index]
            high_price = high_prices[index]  # Use High price
            low_price = low_prices[index]    # Use Low price
            index += 1
        else:
            print("End of historical price data.")
            break

        # Extract Close Prices for SMA Calculation (take the last `sma_long_period` prices)
        if index >= sma_long_period:
            recent_prices = close_prices[index - sma_long_period:index]

            # Calculate Moving Averages
            sma_short = np.mean(recent_prices[-sma_short_period:])
            sma_long = np.mean(recent_prices)

            # Calculate RSI using Close prices
            if index >= sma_long_period + rsi_period:
                rsi_values = calculate_rsi(close_prices[index - (sma_long_period + rsi_period):index], rsi_period)
                rsi = rsi_values[-1]
            else:
                continue

            # Debug information for moving averages and RSI
            print(f"SMA Short: {sma_short}, SMA Long: {sma_long}, RSI: {rsi}")

            # Buy Signal with Relaxed RSI Confirmation
            if sma_short > sma_long and current_position is None and rsi < 40:
                qty = int(cash // current_price)
                if qty > 0:
                    stock_qty = qty
                    cash -= stock_qty * current_price
                    entry_price = current_price
                    current_position = 'long'
                    print(f"Bought {stock_qty} shares at {current_price}, Remaining Cash: {cash}")

            # Stop-loss or Take-profit check based on High and Low prices
            if current_position == 'long':
                # Stop-loss triggered based on Low price
                if low_price <= entry_price * (1 - stop_loss_pct):
                    cash += stock_qty * low_price
                    profit = (low_price - entry_price) * stock_qty
                    total_profit += profit
                    print(f"Stop-Loss Triggered: Sold {stock_qty} shares at {low_price}, Profit: {profit}, New Cash Balance: {cash}")
                    stock_qty = 0
                    current_position = None

                # Take-profit triggered based on High price
                elif high_price >= entry_price * (1 + take_profit_pct):
                    cash += stock_qty * high_price
                    profit = (high_price - entry_price) * stock_qty
                    total_profit += profit
                    print(f"Take-Profit Triggered: Sold {stock_qty} shares at {high_price}, Profit: {profit}, New Cash Balance: {cash}")
                    stock_qty = 0
                    current_position = None

            # Sell Signal with Relaxed RSI Confirmation
            elif sma_short < sma_long and current_position == 'long' and rsi > 60:
                cash += stock_qty * current_price
                profit = (current_price - entry_price) * stock_qty
                total_profit += profit
                print(f"Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
                stock_qty = 0
                current_position = None

            # Debug profit/loss tracking
            print(f"Total Profit/Loss so far: {total_profit}")

        time.sleep(1)

    # Final evaluation of performance
    if current_position == 'long':
        # Sell any remaining stock at the last available price
        cash += stock_qty * current_price
        profit = (current_price - entry_price) * stock_qty
        total_profit += profit
        print(f"Final Sale: Sold {stock_qty} shares at {current_price}, Profit: {profit}, New Cash Balance: {cash}")
        stock_qty = 0
        current_position = None

    print(f"Total Profit/Loss at the end of simulation: {total_profit}")

trade_finance()


[*********************100%%**********************]  1 of 1 completed


SMA Short: 226.17301025390626, SMA Long: 226.60627975463868, RSI: 39.49120140669107
Total Profit/Loss so far: 0
SMA Short: 226.08000946044922, SMA Long: 226.52127990722656, RSI: 40.57542703984777
Total Profit/Loss so far: 0
SMA Short: 226.08209838867188, SMA Long: 226.43360977172853, RSI: 42.86405270932569
Total Profit/Loss so far: 0
SMA Short: 226.11846771240235, SMA Long: 226.45609436035156, RSI: 44.81758863427945
Total Profit/Loss so far: 0
SMA Short: 226.11346740722655, SMA Long: 226.48459396362304, RSI: 44.06549074588909
Total Profit/Loss so far: 0
SMA Short: 226.19046783447266, SMA Long: 226.50834426879882, RSI: 41.02472385579424
Total Profit/Loss so far: 0
SMA Short: 226.4159683227539, SMA Long: 226.57909469604493, RSI: 43.087974255913224
Total Profit/Loss so far: 0
SMA Short: 226.58645782470703, SMA Long: 226.6353446960449, RSI: 41.48751418279277
Total Profit/Loss so far: 0
SMA Short: 226.8124572753906, SMA Long: 226.71284408569335, RSI: 39.2426738383142
Bought 439 shares at 22